In [400]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [401]:
df_runs_lan=pd.read_csv("data/LAN vs SUR - 2023-07-07/2023-07-07-LAN-Batting.csv")
df_wickets_lan=pd.read_csv("data/LAN vs SUR - 2023-07-07/2023-07-07-LAN-FoW.csv")

df_runs_sur=pd.read_csv("data/LAN vs SUR - 2023-07-07/2023-07-07-SUR-Batting.csv")
df_wickets_sur=pd.read_csv("data/LAN vs SUR - 2023-07-07/2023-07-07-SUR-FoW.csv")

df_battingorder=pd.read_csv("data/LAN vs SUR - 2023-07-07/Batting Order.csv")

In [402]:
#Sorting in ascending order by balls
df_runs_lan.sort_values(by="ball", ascending=True, inplace=True)
df_runs_lan=df_runs_lan.reset_index(drop=True)

df_runs_sur.sort_values(by="ball", ascending=True, inplace=True)
df_runs_sur=df_runs_sur.reset_index(drop=True)

In [403]:
# Adding Team Labels
df_runs_lan["Team"]="Lancashire"
df_runs_sur["Team"]="Surrey"

In [404]:
#Sorting Wickets Dataset
df_wickets_lan.sort_values(by="ball", ascending=True, inplace=True)
df_wickets_lan=df_wickets_lan.reset_index(drop=True)

df_wickets_sur.sort_values(by="ball", ascending=True, inplace=True)
df_wickets_sur=df_wickets_sur.reset_index(drop=True)

In [405]:
#Joining on Wickets and Runs datasets
df_lan = df_runs_lan.merge(
    df_wickets_lan,
    on=["ball", "score", "commentary"],
    how="left"
)

df_sur = df_runs_sur.merge(
    df_wickets_sur,
    on=["ball", "score", "commentary"],
    how="left"
)

In [406]:
df = pd.concat([df_lan, df_sur], axis=0, ignore_index=True)
df.shape

(249, 5)

In [407]:
#Scores Cleaning
#Replace "." with "0"
df['score'] = df['score'].replace("•", 0)

#Extracting Runs or Numeric elements from the Scores column
df["Runs"]=None
df['Runs'] = df['score'].astype(str).str.extract('(\d+)')
df['Runs'] = df['Runs'].fillna(0).astype(int)

<>:7: SyntaxWarning: invalid escape sequence '\d'
<>:7: SyntaxWarning: invalid escape sequence '\d'
/var/folders/0r/jdkpl6hs7m184t4p3zcgbsv00000gn/T/ipykernel_39872/3216542835.py:7: SyntaxWarning: invalid escape sequence '\d'
  df['Runs'] = df['score'].astype(str).str.extract('(\d+)')


In [408]:
#On-Strike batter
df["commentary_split"] = df["commentary"].str.split(",")
df["bowler_to_batter"] = df["commentary_split"].str[0]
df["result"] = df["commentary_split"].str[1]
df["bowler_to_batter_split"] = df["bowler_to_batter"].str.split(" to ")
df["bowler"] = df["bowler_to_batter_split"].str[0]
df["on_strike_batter"] = df["bowler_to_batter_split"].str[1]

In [409]:
#Data Specific
df['on_strike_batter'] = df['on_strike_batter'].apply(
    lambda x: "Curran" if x == "Sam Curran" else x
)

In [410]:
# convert the 'ball' column to a string
df['ball'] = df['ball'].astype(str)

# Split 'ball' into 'Over' and 'Ball' columns
df[['Over', 'Ball']] = df['ball'].str.split('.', expand=True)

# Convert 'Over' and 'Ball' columns to integers
df['Over'] = df['Over'].astype(int)
df['Ball'] = df['Ball'].astype(int)

In [411]:
# Creating Out Batter details
df['out_batsman'] = None

# Iterate over rows to apply condition and extraction
for i in range(len(df)):
    # Check if 'fow' is a non-null and non-empty string
    if isinstance(df["fow"][i], str) and df["fow"][i]:
        
        df.loc[i, 'out_batsman'] = df["fow"][i].split(' ')[1] if len(df["fow"][i].split(' ')) > 1 else None

In [412]:
#check
df[df["fow"].isna()==False].shape

(11, 15)

In [413]:
df = df.merge(
    df_battingorder,
    left_on=["Team","on_strike_batter"],
    right_on=["Team","Batter"],
    how="left"
)

In [414]:
df = df.merge(
    df_battingorder,
    left_on=["Team","out_batsman"],
    right_on=["Team","Batter"],
    how="left"
)



In [415]:
df=df.drop(["Batter_x","Batter_y"],axis=1)
df = df.rename(columns={'Batting Order_x': 'Batting_Order_Strike'})
df = df.rename(columns={'Batting Order_y': 'Batting_Order_Out'})
df['Batting_Order_Out'] = df['Batting_Order_Out'].astype('Int64')

In [416]:
def assign_partnership_key(df_team, team_name):
    """
    Assigns a partnership key to batters on pitch for a specific team.
    
    Parameters:
        df_team (pd.DataFrame): Filtered DataFrame for the specific team.
        team_name (str): Name of the team being processed.
        
    Returns:
        pd.DataFrame: DataFrame with partnership keys updated.
    """
    # Reset index for consistent row iteration
    df_team.reset_index(drop=True, inplace=True)

    # Initialize partnership_key and counter
    partnership_key = [1, 2]
    counter = 3

    # Ensure the column can hold strings or tuples
    df_team["partnership_key"] = None  # Initialize the column

    for i in range(len(df_team)):
        if pd.isna(df_team.iloc[i]["Batting_Order_Out"]):  # Check if Batting_Order_Out is NaN
            df_team.at[i, "partnership_key"] = tuple(partnership_key)
        else:
            current_min = min(partnership_key)
            current_max = max(partnership_key)

            if df_team.iloc[i]["Batting_Order_Out"] == current_min:
                df_team.at[i, "partnership_key"] = tuple(partnership_key)
                new_key = counter
                counter += 1
                partnership_key[partnership_key.index(current_min)] = new_key
            else:
                df_team.at[i, "partnership_key"] = tuple(partnership_key)
                new_key = counter
                counter += 1
                partnership_key[partnership_key.index(current_max)] = new_key

    print(f"Processed team: {team_name}")
    return df_team

# Process each team using the function
df_lancashire = assign_partnership_key(df[df["Team"] == "Lancashire"].copy(), "Lancashire")
df_surrey = assign_partnership_key(df[df["Team"] == "Surrey"].copy(), "Surrey")

# Combine the results back into the main DataFrame
df = pd.concat([df_lancashire, df_surrey], axis=0)


Processed team: Lancashire
Processed team: Surrey


In [417]:
# Add a new column with the number in the list but not in 'single_number'
df['Batting_Order_NonStrike'] = df.apply(
    lambda row: [num for num in row['partnership_key'] if num != row['Batting_Order_Strike']][0],
    axis=1
)

In [418]:
df = df.merge(
    df_battingorder,
    left_on=["Team","Batting_Order_NonStrike"],
    right_on=["Team","Batting Order"],
    how="left"
)

df = df.rename(columns={'Batter': 'non_strike_batter'})

In [419]:
# Split the tuples into separate columns
df_split = df['partnership_key'].apply(pd.Series)

# Assign meaningful column names (if you know the structure of the tuple)
df_split.columns = ['Partner1', 'Partner2']  # Replace with actual names if needed

# Concatenate the new columns back to the original DataFrame
df = pd.concat([df, df_split], axis=1)

In [420]:
#Names for Partner1 and Partner 2
df = df.merge(
    df_battingorder,
    left_on=["Team","Partner1"],
    right_on=["Team","Batting Order"],
    how="left"
)

In [421]:
#Names for Partner2
df = df.merge(
    df_battingorder,
    left_on=["Team","Partner2"],
    right_on=["Team","Batting Order"],
    how="left"
)

In [422]:
# Rename the column
df = df.rename(columns={'Batter_x': 'Partner_1',"Batter_y":'Partner_2'})

In [423]:
df.head()

,ball,score,commentary,Team,fow,Runs,commentary_split,bowler_to_batter,result,bowler_to_batter_split,...,partnership_key,Batting_Order_NonStrike,non_strike_batter,Batting Order_x,Partner1,Partner2,Partner_1,Batting Order_y,Partner_2,Batting Order
0,0.1,1,"Jacks to Salt, 1 run",Lancashire,NaN,1,"[Jacks to Salt, 1 run]",Jacks to Salt,1 run,"[Jacks, Salt]",...,"(1, 2)",2,Buttler,2,1,2,Salt,1,Buttler,2
1,0.2,1,"Jacks to Buttler, 1 run",Lancashire,NaN,1,"[Jacks to Buttler, 1 run]",Jacks to Buttler,1 run,"[Jacks, Buttler]",...,"(1, 2)",1,Salt,1,1,2,Salt,1,Buttler,2
2,0.3,4,"Jacks to Salt, FOUR runs",Lancashire,NaN,4,"[Jacks to Salt, FOUR runs]",Jacks to Salt,FOUR runs,"[Jacks, Salt]",...,"(1, 2)",2,Buttler,2,1,2,Salt,1,Buttler,2
3,0.4,1,"Jacks to Salt, 1 run",Lancashire,NaN,1,"[Jacks to Salt, 1 run]",Jacks to Salt,1 run,"[Jacks, Salt]",...,"(1, 2)",2,Buttler,2,1,2,Salt,1,Buttler,2
4,0.5,6,"Jacks to Buttler, SIX runs",Lancashire,NaN,6,"[Jacks to Buttler, SIX runs]",Jacks to Buttler,SIX runs,"[Jacks, Buttler]",...,"(1, 2)",1,Salt,1,1,2,Salt,1,Buttler,2


In [424]:
#Partner Name List
df['partnership_key_name'] = df.apply(lambda row: [row['Partner_1'], row['Partner_2']], axis=1)

In [425]:
df['partnership_key_name'] = df['partnership_key_name'].apply(tuple)


result = df.groupby(['partnership_key_name', 'partnership_key'])['Runs'].sum()


result_reset = result.reset_index()

# Sort by 'partnership_key'
result_sorted = result_reset.sort_values(by='partnership_key', ascending=True)
result_sorted =result_sorted[["partnership_key_name","Runs"]]


result_sorted = result_sorted.reset_index()

print(result_sorted[["partnership_key_name","Runs"]])


    partnership_key_name  Runs
0         (Evans, Jacks)    32
1        (Salt, Buttler)    35
2           (Evans, Roy)    72
3       (Croft, Buttler)    62
4   (Croft, Livingstone)    27
5      (Croft, Mitchell)    35
6          (Curran, Roy)    36
7      (Curran, Overton)    18
8      (Jordan, Overton)    18
9      (Wells, Mitchell)     2
10      (Jordan, Narine)    11
11     (Vilas, Mitchell)     7
12     (Jones, Mitchell)     6


In [426]:
#Extras Runs Treatments
df["nb_runs"] = df["score"].apply(lambda x: 2 if "n" in str(x) else 0)

df["wide_runs"] = df["score"].apply(lambda x: 1 if "w" in str(x) else 0)

df["leg_bye_flag"] = df["commentary"].apply(lambda x: 1 if "leg bye" in str(x) else 0)

df["bye_flag"] = df.apply(lambda row: 1 if "byes" in str(row["commentary"]) and "lb" not in str(row["score"]) else 0, axis=1)

df["leg_bye_runs"] = df.apply(lambda row: row["Runs"] - row["nb_runs"] - row["wide_runs"] 
                              if row["leg_bye_flag"] == 1 else 0, axis=1)

df["bye_runs"] = df.apply(lambda row: row["Runs"] - row["nb_runs"] - row["wide_runs"] 
                              if row["bye_flag"] == 1 else 0, axis=1)

df["extras_runs"]=df["nb_runs"]+df["wide_runs"]+df["leg_bye_runs"]+df["bye_runs"]

In [427]:
#Runs of the bat- total runs - extras
df["Bat_Runs"]=df["Runs"]-df["extras_runs"]

In [428]:
#Generate Ball ID
df['Ball ID'] = df.groupby('Team').cumcount()

In [429]:
df=df[['Team', 'Over', 'Ball', 'Ball ID', 'score', 'Runs', 'bowler', 
    'on_strike_batter', 'commentary', 'fow', 'Batting_Order_Strike', 
    'out_batsman', 'Batting_Order_Out', 'partnership_key', 
    'Batting_Order_NonStrike', 'non_strike_batter', 
    'partnership_key_name', 'nb_runs', 'wide_runs', 'leg_bye_flag', 
    'bye_flag', 'leg_bye_runs', 'bye_runs', 'extras_runs', 'Bat_Runs']]

In [430]:
df=df.reset_index(drop=True)

In [431]:
df.to_csv("output/2023-07-07-FinalOutputData.csv")